In [10]:
from googleapiclient.discovery import build
from datetime import datetime, timedelta
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import numpy as np
import pandas as pd
import isodate
import requests
import time
from selenium import webdriver
from IPython.display import JSON

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
# api key load
with open('api.txt', 'r') as file:
    api_key = file.read().strip()

api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(api_service_name, api_version, developerKey=api_key)

In [3]:
# URL 모음
base_url = "https://store.steampowered.com/"

url_stats = 'stats/'
url_search = 'search/'
url_category = 'category/'

#url_filter = '?filter='
url_filter = '?flavor=contenthub_newandtrending'

# offset
url_offset = '&offset='

# 스팀 장르별 URL 모음
url_action = 'action/'
url_action_fps = 'action_fps/'
url_action_tps = 'action_tps/'
url_fighting_marial_arts = 'fighting_marial_arts/'
url_shump = 'shump/'
url_arcade_rhythm = 'arcade_rhythm/'
url_action_run_jump = 'action_run_jump/'
url_hack_and_slash = 'hack_and_slash/'

url_rpg = 'rpg/'
url_jrpg = 'rpg_jrpg/'
url_rogue_like_rogue_lite = 'rogue_like_rogue_lite/'
url_rpg_action = 'rpg_action/'
url_adventure_rpg = 'adventure_rpg/'
url_rpg_strategy = 'rpg_strategy_tactics/'
url_rpg_turn_based = 'rpg_turn_based/'
url_rpg_party_based = 'rpg_party_based/'

url_strategy = 'strategy/'
url_strategy_military = 'strategy_military/'
url_strategy_grand_4x = 'strategy_grand_4x/'
url_strategy_cities_settlements = 'strategy_cities_settlements/'
url_strategy_real_time = 'strategy_real_time/'
url_strategy_card_board = 'strategy_card_board/'
url_tower_defense = 'tower_defense/'
url_strategy_turn_based = 'strategy_turn_based/'

url_adventure = 'adventure/'
url_metroidvania = 'metroidvania/'
url_visual_novel = 'visual_novel/'
url_adventure_rpg = 'adventure_rpg/'
url_casual = 'casual/'
url_puzzle_matching = 'puzzle_matching/'
url_story_rich = 'story_rich/'
url_hidden_object = 'hidden_object/'

rul_simulation = 'simulation/'
rul_sim_building_automation = 'sim_building_automation/'
rul_sim_farming_crafting = 'sim_farming_crafting/'
rul_sim_physics_sandbox = 'sim_physics_sandbox/'
rul_sim_life = 'sim_life/'
rul_sim_dating = 'sim_dating/'
rul_sim_space_flight = 'sim_space_flight/'
rul_sim_hobby_sim = 'sim_hobby_sim/'

url_sports_and_racing = 'sports_and_racing/'
url_sports_individual = 'sports_individual/'
url_sports_fishing_hunting = 'sports_fishing_hunting/'
url_racing = 'racing/'
url_racing_sim = 'racing_sim/'
url_sports = 'sports/'
url_sports_sim = 'sports_sim/'
url_sports_team = 'sports_team/'

## 테마 별 URL
url_science_fiction = 'science_fiction/'
url_horror = 'horror/'
url_mystery_detective = 'mystery_detective/'
url_survival = 'survival/'
url_adultonly = 'adultonly/'
url_anime = 'anime/'
url_exploration_open_world = 'exploration_open_world/'
url_space = 'space/'

## 플레이어 지원 별 URL
url_multiplayer_lan = 'multiplayer_lan/'
url_multiplayer_mmo = 'multiplayer_mmo/'
url_multiplayer_local_party = 'multiplayer_local_party/'
url_multiplayer = 'multiplayer/'
url_singleplayer = 'singleplayer/'
url_multiplayer_online_competitive = 'multiplayer_online_competitive/'
url_multiplayer_coop = 'multiplayer_coop/'

## URL 합치기
use_url = base_url + url_category + url_multiplayer + url_filter

In [4]:
def new_popular_game_scan(use_URL):
    # Selenium 옵션 설정
    options = Options()
    options.headless = True

    # Chrome Driver 실행
    service = Service(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    
    driver.get(use_URL)
    
    game_data = []
    time.sleep(10)
    games = driver.find_elements(By.CLASS_NAME, "_1lRFu670LVk6Gmeb12h7Hr")
    time.sleep(10)
    
    # 게임 정보 리스트에 추가
    for game in games:
        name = game.find_element(By.CLASS_NAME, "_3jI467XYJLy1CQ5YZhp2q_").text.strip()
        time.sleep(5)
        price = game.find_element(By.CLASS_NAME, "Wh0L8EnwsPV_8VAu8TOYr").text.strip()
        release_date = game.find_element(By.CLASS_NAME, "_3eOdkTDYdWyo_U5-JPeer1").text.strip()
        tags = game.find_elements(By.CLASS_NAME, "WidgetTag")

        tag_list = [tag.text for tag in tags]
        tag_list += [''] * (5 - len(tag_list))

        # 게임 데이터에 추가
        game_data.append([name, price, release_date] + tag_list[:5])

    # 데이터프레임 생성
    new_games = pd.DataFrame(game_data, columns=['게임 이름', '가격', '출시일', 'Tag 1', 'Tag 2', 'Tag 3', 'Tag 4', 'Tag 5'])
    driver.quit()
    
    return new_games

In [33]:
df_list = []

for i in range(0, 5):
    offset = str(12 * i)
    use_url = base_url + url_category + url_multiplayer + url_filter + url_offset + offset
    
    temp = new_popular_game_scan(use_url)
    df_list.append(temp)
    time.sleep(10)
    
df_combined = pd.concat(df_list, ignore_index=True)
df_combined

,게임 이름,가격,출시일,Tag 1,Tag 2,Tag 3,Tag 4,Tag 5
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,Backpack Battles,"₩ 13,050","MAR 8, 2024",Auto Battler,Strategy,Inventory Management,PvP,Multiplayer
...,...,...,...,...,...,...,...,...
74,New Star GP,"₩ 32,000","MAR 8, 2024",Racing,Sports,Driving,Arcade,Multiplayer
75,Trinity Survivors,"₩ 5,500","JAN 31, 2024",Hack and Slash,RPG,Roguelite,PvE,Roguelike
76,唱舞全明星,Free To Play,"MAR 14, 2024",RPG,Rhythm,Action,Casual,Character Customization
77,Broken World,"₩ 3,400","MAR 7, 2024",Survival,Open World,Indie,Exploration,Sandbox


In [40]:
df_list[4]

,게임 이름,가격,출시일,Tag 1,Tag 2,Tag 3,Tag 4,Tag 5
0,,,,,,,,
1,,,,,,,,
2,,,,,,,,
3,,,,,,,,
4,Brothers: A Tale of Two Sons Remake,"₩ 20,400","FEB 29, 2024",Adventure,Casual,Local Co-Op,Action-Adventure,Singleplayer
5,Smalland: Survive the Wilds,"₩ 37,500","FEB 15, 2024",Survival,Open World Survival Craft,Multiplayer,Base Building,Online Co-Op
6,Bunny Game,"₩ 3,840","MAR 12, 2024",Social Deduction,Multiplayer,Online Co-Op,3D,Action-Adventure
7,Windowkill,"₩ 5,600","FEB 24, 2024",Action,Action Roguelike,Bullet Hell,Arcade,Twin Stick Shooter
8,Kingdom: The Blood,Free To Play,"MAR 5, 2024",Action,Zombies,Character Customization,Action RPG,Online Co-Op
9,Find Cats With Friends,₩ 660,"MAR 14, 2024",Casual,Simulation,Walking Simulator,Creature Collector,3D


In [44]:
df3 = df_combined.replace('', np.nan)
data = df3.dropna()
data = data.reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   게임 이름   60 non-null     object
 1   가격      60 non-null     object
 2   출시일     60 non-null     object
 3   Tag 1   60 non-null     object
 4   Tag 2   60 non-null     object
 5   Tag 3   60 non-null     object
 6   Tag 4   60 non-null     object
 7   Tag 5   60 non-null     object
dtypes: object(8)
memory usage: 3.9+ KB


In [45]:
# new_popular_game_data.to_csv('new popular game Top 12.csv', encoding="utf-8-sig", index=False)
data.to_csv('multiplayer_new_popular_games.csv', encoding="utf-8-sig", index=False)

In [149]:
# Selenium 옵션 설정
options = Options()
options.headless = True

# Chrome Driver 실행
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 인기 있는 게임 페이지로 이동
driver.get('https://store.steampowered.com/tag/browse#global_492')

# 게임 정보 가져오기
game_data = []

# driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10)
elements = driver.find_elements(By.CLASS_NAME, "tag_browse_tag")

tag_list = []
# 각 요소의 텍스트를 추출하고 출력합니다.
for element in elements:
    tag_list.append(element.text)

time.sleep(10)
# 게임 정보 리스트에 추가
# for game in games:
#     name = game.find_element(By.CLASS_NAME, "_3jI467XYJLy1CQ5YZhp2q_").text.strip()
#     tags = game.find_element(By.CLASS_NAME, "_3OSJsO_BdhSFujrHvCGLqV").text.strip()
#     price = game.find_element(By.CLASS_NAME, "Wh0L8EnwsPV_8VAu8TOYr").text.strip()
#     release_date = game.find_element(By.CLASS_NAME, "_3eOdkTDYdWyo_U5-JPeer1").text.strip()
#     game_data.append([name, price, release_date, tags])

# 데이터프레임 생성
# new_games = pd.DataFrame(game_data, columns=['게임 이름', '가격', '출시일', '연관 태그'])
# driver.quit()

In [150]:
tag_list

['Indie',
 'Action',
 'Adventure',
 'Casual',
 'RPG',
 'Simulation',
 'Strategy',
 'Singleplayer',
 'Early Access',
 'Free to Play',
 '2D',
 '3D',
 'Atmospheric',
 'Fantasy',
 'Colorful',
 'Story Rich',
 'Multiplayer',
 'Puzzle',
 'Exploration',
 'Pixel Graphics',
 'Cute',
 'Massively Multiplayer',
 'Sports',
 'Combat',
 'First-Person',
 'Violent',
 'Action-Adventure',
 'Anime',
 'Arcade',
 'Funny',
 'Racing',
 'Sexual Content',
 'Sci-fi',
 'Nudity',
 'Shooter',
 'Relaxing',
 'Horror',
 'Family Friendly',
 'Third Person',
 'Controller',
 'Retro',
 'Gore',
 'Stylized',
 'Female Protagonist',
 'Open World',
 'Co-op',
 'Survival',
 'Platformer',
 'PvP',
 'Top-Down',
 'Choices Matter',
 'Realistic',
 'PvE',
 'Comedy',
 'Dark',
 'Visual Novel',
 'Character Customization',
 'Cartoony',
 'Linear',
 'Mystery',
 'Physics',
 'Multiple Endings',
 '2D Platformer',
 'Online Co-Op',
 'FPS',
 'Old School',
 'Psychological Horror',
 'Magic',
 'Difficult',
 'Sandbox',
 'Tactical',
 'Medieval',
 'VR',
 

In [136]:
game_names = new_popular_game_data['게임 이름'].tolist()
game_names

['Backpack Battles',
 'Dune: Imperium',
 'WWE 2K24',
 'The Outlast Trials',
 'Last Epoch',
 'Sons Of The Forest',
 'Stolen Realm',
 'TRIBES 3: Rivals',
 'HELLDIVERS™ 2',
 'Contra: Operation Galuga',
 'Granblue Fantasy: Relink',
 'Myth of Empires']

In [142]:
def search_new_games(game_names):
    video_results = []
    for game_name in game_names:
        request = youtube.search().list(
            q=game_name,
            part='id,snippet',
            maxResults=5,
            type='video'
        )
        response = request.execute()
        videos = response['items']
        for video in videos:
            video_id = video['id']['videoId']
            video_request = youtube.videos().list(
                part="snippet,contentDetails,statistics",
                id=video_id
            )
            video_response = video_request.execute()
            video_info = video_response['items'][0]
            video_results.append({
                'Game': game_name,
                'Title': video_info['snippet']['title'],
                'Video ID': video_id,
                'Channel': video_info['snippet']['channelTitle'],
                'Duration': video_info['contentDetails']['duration'],
                'Definition': video_info['contentDetails']['definition'],
                'Caption': video_info['contentDetails'].get('caption', ''),
                'View Count': video_info['statistics']['viewCount'],
                'Like Count': video_info['statistics']['likeCount'],
                'Favorite Count': video_info['statistics']['favoriteCount'],
                'Comment Count': video_info['statistics']['commentCount']
            })

    # 데이터프레임으로 변환
    video_df = pd.DataFrame(video_results)
    return video_df

In [143]:
def change_time(df, col):
    df[col] = df[col].apply(lambda x: isodate.parse_duration(x).total_seconds())

    return df

In [144]:
new_game_videos = search_new_games(game_names)

In [145]:
change_time(new_game_videos, 'Duration')

,Game,Title,Video ID,Channel,Duration,Definition,Caption,View Count,Like Count,Favorite Count,Comment Count
0,Backpack Battles,Full Beginner Guide + Tips for Backpack Battles,q6LqxFtlDVQ,Ben's Backpack,998.0,hd,true,26089,600,0,23
1,Backpack Battles,The New LUCK CLASS is INSANE! | Backpack Battles,ldUy-Y7jy2Y,Sifd,2318.0,hd,false,27085,770,0,56
2,Backpack Battles,Gotta Catch 'Em All! (Dragon Pyromancer) | Bac...,u7NUf5vEfmg,Ben's Backpack,1940.0,hd,false,9759,310,0,45
3,Backpack Battles,"This Build WILL Get Nerfed, its TOO OP!! | Bac...",3mjc2IzMNpg,S〽ojo,1406.0,hd,false,272,13,0,2
4,Backpack Battles,ОСТОРОЖНО! Вызывает привыкание. Лучший Обзор B...,ve_9Pv3crCw,Дуда,620.0,hd,false,4827,271,0,140
5,Dune: Imperium,Dune: Imperium - How To Play,nvX9EYxQEfY,Watch It Played,1863.0,hd,true,505966,10653,0,731
6,Dune: Imperium,How to Play Dune Imperium (Digital!) - Full Pl...,hnal3EKdNGo,BLUDGEON,7654.0,hd,false,2368,73,0,25
7,Dune: Imperium,Dune Imperium in about 3 minutes,wER9hInn92A,3 Minute Board Games,235.0,hd,true,70517,1370,0,84
8,Dune: Imperium,Dune: Imperium Review - A SU&SD Dust-Up,kiXL251bdmQ,Shut Up & Sit Down,1075.0,hd,false,293334,7659,0,683
9,Dune: Imperium,Dune Imperium: Uprising (And CHOAM Module) - H...,3cwzF63F0ks,Watch It Played,2852.0,hd,false,71121,1475,0,172


In [146]:
new_game_videos.to_csv('new_game_data.csv', index=False)